In [1]:
!nvidia-smi

Tue Jul 23 15:22:55 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:87:00.0 Off |                    0 |
| N/A   42C    P8    15W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

<center>
  
# TABSYN: Tabular Data Synthesis with Diffusion Models

</center>

Two challenges regarding the extention of diffusion models to tabular data are:
1. **Diverse data types:** a single table can have different columns each containing data of different types, including numerical, categorical, text, etc.
2. **Varied distributions:** the distribution of data under different columns in a single table varry widely from column to column.

**TabSyn** addresses these challenges by introducing a latent space where tabular data of all columns are jointly represented. It then proceedes to train a diffusion model on the latent representations.
This tactic allows TabSyn to:
1. Train a single diffusion model for all data types in the dataset (i.e. Generality).
2. Optimize the distribution of latent embeddings to facilitate training of the subsequent diffusion model, thus generating higher quality synthetic data (i.e. Quality).
3. Require much fewer reverse steps during training of the diffusion model, and synthesize data faster (i.e. Speed).

In this notebook, we review and implement the TabSyn model.


# Imports and Setup

In this section, we import all necessary libraries and modules required for setting up the environment.
Most of the libraries we need to implement TabSyn are the same as TabDDPM.
We also specify `NAME_URL_DICT_UCI`, `DATA_NAME`, `DATA_DIR` and other paths as in TabDDPM's implementation.


In [3]:
import os
import src
import json
import pandas as pd
from pprint import pprint

import torch
from torch.utils.data import DataLoader

from scripts.download_dataset import download_from_uci
from scripts.process_dataset import process_data

from src.data import preprocess, TabularDataset
from src.baselines.tabsyn.pipeline import TabSyn


NAME_URL_DICT_UCI = {
    "adult": "https://archive.ics.uci.edu/static/public/2/adult.zip",
    "default": "https://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip",
    "magic": "https://archive.ics.uci.edu/static/public/159/magic+gamma+telescope.zip",
    "shoppers": "https://archive.ics.uci.edu/static/public/468/online+shoppers+purchasing+intention+dataset.zip",
    "beijing": "https://archive.ics.uci.edu/static/public/381/beijing+pm2+5+data.zip",
    "news": "https://archive.ics.uci.edu/static/public/332/online+news+popularity.zip",
}

DATA_DIR = "/projects/aieng/diffusion_bootcamp/data/tabular_copy"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
SYNTH_DATA_DIR = os.path.join(DATA_DIR, "synthetic_data")
DATA_NAME = "adult"

MODEL_PATH = "/projects/aieng/diffusion_bootcamp/models/tabular/tabsyn_copy"

# Adult Dataset

In this section, we will download the Adult dataset from the UCI repository and load it into a pandas DataFrame.
The Adult dataset contains demographic information about individuals, such as age, education, and occupation, and is commonly used for classification tasks.
We will use this dataset to demonstrate the TabSyn method.
For more explanation of different steps in this section, please refer to TabDDPM's notebook.

In [4]:
# download data
download_from_uci(DATA_NAME, RAW_DATA_DIR, NAME_URL_DICT_UCI)

# process data
INFO_DIR = "data_info"
process_data(DATA_NAME, INFO_DIR, DATA_DIR)

# review data
df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "train.csv"))
df.head(5)

Start processing dataset adult from UCI.
Aready downloaded.
adult (32561, 15) (16281, 15) (32561, 15)
Numerical (32561, 6)
Categorical (32561, 8)
Processing and Saving adult Successfully!
adult
Total 48842
Train 32561
Test 16281
Num 6
Cat 9


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [5]:
# clean data
value = " ?"
if value in df.values:
    print(f"{value} exists in the DataFrame.")
else:
    print(f"{value} does not exist in the DataFrame.")

# review json file and its contents
with open(f"{PROCESSED_DATA_DIR}/{DATA_NAME}/info.json", "r") as file:
    data_info = json.load(file)
pprint(data_info)

 ? exists in the DataFrame.
{'cat_col_idx': [1, 3, 5, 6, 7, 8, 9, 13],
 'column_info': {'0': {},
                 '1': {},
                 '10': {},
                 '11': {},
                 '12': {},
                 '13': {},
                 '14': {},
                 '2': {},
                 '3': {},
                 '4': {},
                 '5': {},
                 '6': {},
                 '7': {},
                 '8': {},
                 '9': {},
                 'categorizes': [' <=50K', ' >50K'],
                 'max': 99.0,
                 'min': 1.0,
                 'type': 'categorical'},
 'column_names': ['age',
                  'workclass',
                  'fnlwgt',
                  'education',
                  'education.num',
                  'marital.status',
                  'occupation',
                  'relationship',
                  'race',
                  'sex',
                  'capital.gain',
                  'capital.loss',
          

# TabSyn Algorithem

In this section, we will describe the design of TabSyn as well as its main hyperparameters loaded through config, which affect the model’s effectiveness. 

**TabSyn** consists of two parts:
1. A *variational auto-encoder (VAE)* which learns a joint representation space for the given tabular data.
2. A *Diffusion model* which learns the distribution of data in the joint representation space.

The figure below shows a diagram of the TabSyn model.

<p align="center">
<img src="figures/tabsyn.jpg" width="1000"/>
</p>

**VAE**

The left-side of the figure shows the VAE which operates in the original data space. The VAE itself consists of two parts: an encoder and a decoder. It also contains the corresponding tokenizer and detokenizer.
Each row of the input tabular data ($\pmb{x}$) is tokenized, then embedded by a transformer. Another transformer decodes the embeddings and a detokenizer reconstructs the table ($\pmb{\tilde{x}}$). The VAE is trained by minimizing the reconstruction loss between $\pmb{x}$ and $\pmb{\tilde{x}}$.

After the VAE is fully trained, the whole data ($\pmb{x}$) is tokenized and embedded. The embedding of each row is flattened to form a 1-dimensional vector $\pmb{z}$.
These 1-dimensional embeddings for all rows are stored on disk, and will later be used to train the diffusion model.

**Diffusion**

The right-side of the figure shows the diffusion model which operates in the latent representation space; in other words, it only *sees* the embeddings obtained by the VAE, not the original tabular data.
The diffusion model can be similarly divided into two parts: a forward process, and a reverse process.

The forward process receives the embedded data points. A single data point is denoted by $\pmb{z_0}$ in the figure. Gaussian noise is incrementally added to the embeddings in numerous incremental steps during the forward process. The number of the steps is denoted by $T$ in the figure. $T$ should be high enough that the distribution of embeddings at step $t=T$ is essentially a standard Gaussian distribution; in other words, the signal-to-noise ratio is practically zero.

The reverse process, on the other hand, learns to *predict* an earlier-step embedding (e.g. $\pmb{z_{t-\Delta t}}$) from a later-step embedding (e.g. $\pmb{z_t}$) via a neural network.

After the diffusion model is fully trained, the reverse process can estimate the data distribution at step $t=0$ if it receives a standard Gaussian distribution at step $t=T$. New data points can be synthesized by sampling from this estimated distribution.


**Mathematical Formulation**

Let's formulate the VAE and the diffusion model which we explained above.

## Load Config

In this section, we will load the configuration file that contains the hyperparameters for the TabSyn model. 

In [5]:
config_path = os.path.join("src/baselines/tabsyn/configs", f"{DATA_NAME}.toml")
raw_config = src.load_config(config_path)

pprint(raw_config)

{'loss_params': {'lambd': 0.7, 'max_beta': 0.01, 'min_beta': 1e-05},
 'model_params': {'d_token': 4, 'factor': 32, 'n_head': 1, 'num_layers': 2},
 'sample': {'steps': 50},
 'task_type': 'binclass',
 'train': {'diffusion': {'batch_size': 4096,
                         'num_dataset_workers': 4,
                         'num_epochs': 10001},
           'optim': {'diffusion': {'factor': 0.9,
                                   'lr': 0.001,
                                   'patience': 20,
                                   'weight_decay': 0},
                     'vae': {'factor': 0.95,
                             'lr': 0.001,
                             'patience': 10,
                             'weight_decay': 0}},
           'vae': {'batch_size': 4096,
                   'num_dataset_workers': 4,
                   'num_epochs': 4000}},
 'transforms': {'cat_encoding': None,
                'cat_min_frequency': None,
                'cat_nan_policy': None,
                'normalizat

The configuration file is a TOML file that contains the following hyperparameters:

1. **model_params:** specifies the structure of the transformers (both encoder and decoder) in the VAE model, including number of transformer layers, number of self-attnetion heads, `d_token` and `factor`.

2. **transforms:** specifies the transformations and preprocessing of the data before tokenization, such as cleaning, normalization, encoding and `y_policy`.
    - For preprocessing numerical features, we use the gaussian quantile transformation and replace the NaN values with mean of each row.
    - For categorical features, NaN values are not changed. Do we use one-hot encoding here?
    - What is `y_policy`?

3. **train.vae:** specifies training parameters of the VAE, including batch size, number of epochs, and number of dataset workers.

4. **train.diffusion:** specifies the same training parameters as above for the diffusion model.

5. **train.optim.vae:** specifies the parameters of the Adam optimizer and the `ReduceLROnPlateau` learning rate scheduler used to train the VAE. Optimizer parameters include initial learning rate and weight decay. LR scheduler parameters includer `factor` and `patience`.

6. **train.optim.diffusion:** specifies the same parameters as above for the diffusion model.

6. **loss_params:** specifies parameters of the loss function used to train the VAE including `max_beta`, `min_beta` and `lambd`.

7. **sample:** contains the parameters for sampling the data from the trained diffusion model.


## Make Dataset

Now that we have processed the data, we can create a dataset object. First we instantiate transformations needed for the dataset, such as normalization and cleaning in `transforms`. Next using `preprocess` function we create arrays that contains the training and test data, as well as the number of categries for each categorical feature and the dimension of each numerical feature.

We then separate the train and test data as different arrays and convert them to torch tensors. We create a dataset object with the train data.
We keep the test data as tensors and only move them to the GPU (if available) to be processed later with the trained model.

In [ ]:
# preprocess data
X_num, X_cat, categories, d_numerical = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                   transforms = raw_config["transforms"],
                                                   task_type = raw_config["task_type"])

# separate train and test data
X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

# convert to float tensor
X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

# create dataset module
train_data = TabularDataset(X_train_num.float(), X_train_cat)

# move test data to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_test_num = X_test_num.float().to(device)
X_test_cat = X_test_cat.to(device)

# create train dataloader
train_loader = DataLoader(
    train_data,
    batch_size = raw_config["train"]["vae"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["vae"]["num_dataset_workers"],
)

In [7]:
print("train dataset num shape: ", train_data.X_num.shape)
print("test dataset num shape: ", X_test_num.shape)

print("train dataset cat shape: ", train_data.X_cat.shape)
print("test dataset cat shape: ", X_test_cat.shape)

train dataset num shape:  torch.Size([32561, 6])
test dataset num shape:  torch.Size([16281, 6])
train dataset cat shape:  torch.Size([32561, 9])
test dataset cat shape:  torch.Size([16281, 9])


## Instantiate Model

Next, we instantiate the model using the `TabSyn` class. `TabSyn` class takes the following arguments:

1. `train_loader`
2. `X_test_num`
3. `X_test_cat`
4. `num_numerical_features`
5. `num_classes`
6. `device`

In [8]:
tabsyn = TabSyn(train_loader,
                X_test_num, X_test_cat,
                num_numerical_features = d_numerical,
                num_classes = categories,
                device = device)

`TabSyn` class has tools to instantiate the VAE and the diffusion model, train the VAE, train the diffusion model, and sample from the trained diffusion model.
We will demonstrate how to use these tools in the following sections.

## Train Model


We start with the VAE.


### Train VAE

First, we need to instantiate the VAE using `load_vae_model` method. This method takes the VAE model hyperparameters, optimizer and lr scheduler parameters from config, and instantiates them.

In [9]:
# laod and prepare VAE model for training
tabsyn.load_vae_model(**raw_config["model_params"], optim_params = raw_config["train"]["optim"]["vae"])

self.category_embeddings.weight.shape=torch.Size([104, 4])
self.category_embeddings.weight.shape=torch.Size([104, 4])
Successfully loaded VAE model.


Now that we have instantiated the VAE, we can train it using the `train_vae` function. loss parameters and number of epochs are given from the config.
`model_save_path`, `encoder_save_path` and `decoder_save_path` determine the paths where the weights of the VAE model is saved.

In [ ]:
tabsyn.train_vae(**raw_config["loss_params"],
                 num_epochs = raw_config["train"]["vae"]["num_epochs"],
                 model_save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae", "model.pt"),
                 encoder_save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae", "encoder.pt"),
                 decoder_save_path = os.path.join(MODEL_PATH, DATA_NAME, "vae", "decoder.pt"))

After training the VAE, we embed the training data with the trained encoder and store the embeddings in a direcotry specified by `vae_ckpt_dir`.

In [ ]:
# embed all inputs in the latent space
tabsyn.save_vae_embeddings(X_train_num, X_train_cat,
                           vae_ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME, "vae"))

Successfully saved pretrained embeddings on disk!


### Train Diffusion Model

Now that we have stored the embeddings of the training data, we need to load and prepare them for the training of the diffusion model.
We load the embeddings using `load_vae_embeddings` and normalize them. Then, we create a Dataloader with the specified `batch_size` and `num_workers` from the config.

In [ ]:
# load latent space embeddings
train_z, _ = tabsyn.load_vae_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))  # train_z dim: B x in_dim

# normalize embeddings
mean, std = train_z.mean(0), train_z.std(0)
train_z = (train_z - mean) / 2
latent_train_data = train_z

# create data loader
latent_train_loader = DataLoader(
    latent_train_data,
    batch_size = raw_config["train"]["diffusion"]["batch_size"],
    shuffle = True,
    num_workers = raw_config["train"]["diffusion"]["num_dataset_workers"],
)

Now that the data is ready to train the diffusion model, we instantiate the diffusion model with `load_diffusion_model`. The input dimension and hidden dimention of this model is determined by the dimension of the embeddings. 
We also instantiate the optimizer and lr scheduler using hyperparameters from config.

In [ ]:
# load and prepare diffusion model for training
tabsyn.load_diffusion_model(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = raw_config["train"]["optim"]["diffusion"])

MLPDiffusion(
  (proj): Linear(in_features=60, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=60, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10616892
Successfully loaded diffusion model.


We train the diffusion model with `train_diffusion` function.
This function takes the following arguements:
1. `latent_train_loader`: dataloader for the latent representations which are used to train the diffusion model.
2. `num_epochs`: number of training epochs.
3. `ckpt_path`: directory where the model checkpoints will be stored.

In [ ]:
# train diffusion model
tabsyn.train_diffusion(latent_train_loader,
                       num_epochs = raw_config["train"]["diffusion"]["num_epochs"],
                       ckpt_path = os.path.join(MODEL_PATH, DATA_NAME))

Epoch 9/9: 100%|██████████| 8/8 [00:01<00:00,  6.18it/s, Loss=0.397]


Time:  12.037148237228394


## Load pretrained Model

Instead of training model from scratch, we can also load weights of pre-trained model from given checkpoint with `load_model_state` function.
If we haven't instantiated VAE and diffusion model beforehand, we need to instantiate them first using `load_vae_model` and `load_diffusion_model` methods.

In [ ]:
# import importlib
# import src.baselines.tabsyn.pipeline as pipeline
# importlib.reload(pipeline)
# TabSyn = getattr(pipeline, "TabSyn")

In [ ]:
# laod VAE model
tabsyn.load_vae_model(**raw_config["model_params"], optim_params = None)

# load latent embeddings of input data
train_z, token_dim = tabsyn.load_vae_embeddings(os.path.join(MODEL_PATH, DATA_NAME, "vae"))

# load diffusion model
tabsyn.load_diffusion_model(in_dim = train_z.shape[1], hid_dim = train_z.shape[1], optim_params = None)

# load state from checkpoint
tabsyn.load_model_state(ckpt_dir = os.path.join(MODEL_PATH, DATA_NAME),
                        dif_ckpt_name = "model.pt")

self.category_embeddings.weight.shape=torch.Size([104, 4])
self.category_embeddings.weight.shape=torch.Size([104, 4])
Successfully loaded VAE model.
MLPDiffusion(
  (proj): Linear(in_features=60, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=60, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
The number of parameters: 10616892
Successfully loaded diffusion model.
Loaded model state from /projects/aieng/diffusion_bootcamp/models/tabular/tabsyn_copy/adult


## Sample Data

Now that we trained the model effectively, using `sample` function we can generate synthetic data starting from compelete noise. The input of this function is as follows:

1. `train_z`: latent embeddings of the training data.
2. `info`: info about the data from the jsonl file we reviewed at the beginning of the notebook.
3. `num_inverse`: detokenizer for numerical features.
4. `cat_inverse`: detokenizer for categorical features.
5. `save_path`: filepath where the synthetic table will be saved.

In [ ]:
# load data info file
with open(os.path.join(PROCESSED_DATA_DIR, DATA_NAME, "info.json"), "r") as file:
    data_info = json.load(file)
data_info["token_dim"] = token_dim

# get inverse tokenizers
_, _, categories, d_numerical, num_inverse, cat_inverse = preprocess(os.path.join(PROCESSED_DATA_DIR, DATA_NAME),
                                                                     transforms = raw_config["transforms"],
                                                                     task_type = raw_config["task_type"],
                                                                     inverse = True)

# sample data
tabsyn.sample(train_z,
              info = data_info,
              num_inverse = num_inverse,
              cat_inverse = cat_inverse,
              save_path = os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

data_path /projects/aieng/diffusion_bootcamp/data/tabular_copy/processed_data/adult
No NaNs in numerical features, skipping
(32561, 9)
Time: 15.157551765441895
Saving sampled data to /projects/aieng/diffusion_bootcamp/data/tabular_copy/synthetic_data/adult/tabsyn.csv


## Synthetic Data

Finally here, we review the synthesized data. In the following `evaluate_synthetic_data.ipynb` notebook, we will evaluate this synthesized data with respect to various metrics.

In [ ]:
df = pd.read_csv(os.path.join(SYNTH_DATA_DIR, DATA_NAME, "tabsyn.csv"))

# Display the first few rows of the DataFrame
df.head(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,35.000000,Private,154501.34,HS-grad,6.0,Never-married,Exec-managerial,Wife,Black,Male,0.0,0.0,40.0,Portugal,<=50K
1,36.000000,State-gov,254158.64,Prof-school,9.0,Married-civ-spouse,Adm-clerical,Husband,Black,Male,0.0,0.0,40.0,Portugal,<=50K
2,33.141235,State-gov,187729.34,Assoc-acdm,9.0,Never-married,Exec-managerial,Wife,Black,Male,0.0,0.0,35.0,Portugal,<=50K
3,49.000000,State-gov,165583.39,HS-grad,10.0,Married-civ-spouse,Adm-clerical,Wife,Other,Male,0.0,0.0,40.0,Portugal,>50K
4,44.000000,State-gov,187748.36,Assoc-acdm,9.0,Never-married,Adm-clerical,Wife,Black,Male,0.0,0.0,40.0,Portugal,<=50K


## References

**Zhang, Hengrui, et al.** "Mixed-type tabular data synthesis with score-based diffusion in latent space." *International Conference on Learning Representations (ICLR)* (2023).

**GitHub Repository:** [Amazon Science - Tabsyn](https://github.com/amazon-science/tabsyn)